In [2]:
!pip install datasets
!pip install tensorflow-hub
!pip install sentence-transformers

     |████████████████████████████████| 245kB 5.7MB/s 
     |████████████████████████████████| 245kB 36.2MB/s 
     |████████████████████████████████| 122kB 34.3MB/s 
     |████████████████████████████████| 92kB 4.0MB/s 
     |████████████████████████████████| 2.5MB 27.2MB/s 
     |████████████████████████████████| 1.2MB 44.4MB/s 
     |████████████████████████████████| 901kB 43.3MB/s 
     |████████████████████████████████| 3.3MB 47.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-cp37-none-any.whl size=126711 sha256=b4517e4cd364adb9fd9927fcf0b884a384974f3ea9c2b3b250f4d93d9fed0fa8
  Stored in directory: /root/.cache/pip/wheels/38/d2/98/d191289a877a34c68aa67e05179521e060f96394a3e9336be6
Successfully built sentence-transformers
ERROR: transformers 4.8.2 has requirement huggingface-hub==0.0.12, but you'll have huggingface-hub 0.0.13 which is incompatible.


In [3]:
from datasets import load_metric, load_dataset 
metric = load_metric('glue', 'mrpc') 
mrpc = load_dataset('glue', 'mrpc') 

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


In [4]:
metric = load_metric('glue', 'stsb') 
metric.compute(predictions=[1,2,3],references=[5,2,2]) 

{'pearson': -0.8660254037844387, 'spearmanr': -0.8660254037844387}

In [5]:
from datasets import load_metric, load_dataset 
stsb_metric = load_metric('glue', 'stsb') 
stsb = load_dataset('glue', 'stsb') 

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


In [6]:
import tensorflow_hub as hub 
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4") 
from sentence_transformers import SentenceTransformer 
distilroberta = SentenceTransformer('stsb-distilroberta-base-v2') 

INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.
INFO:absl:Downloaded https://tfhub.dev/google/universal-sentence-encoder/4, Total size: 987.47MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.


In [7]:
import tensorflow as tf 
import math 
def use_sts_benchmark(batch): 
  sts_encode1 = tf.nn.l2_normalize(use_model(tf.constant(batch['sentence1'])),axis=1) 
  sts_encode2 = tf.nn.l2_normalize(use_model(tf.constant(batch['sentence2'])),axis=1) 
  cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1,sts_encode2),axis=1) 
  clip_cosine_similarities = tf.clip_by_value(cosine_similarities,-1.0,1.0)
  scores = 1.0 - tf.acos(clip_cosine_similarities) / math.pi 
  return scores 

In [8]:
def roberta_sts_benchmark(batch): 
  sts_encode1 = tf.nn.l2_normalize(distilroberta.encode(batch['sentence1']),axis=1) 
  sts_encode2 = tf.nn.l2_normalize(distilroberta.encode(batch['sentence2']),axis=1) 
  cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1,sts_encode2),axis=1) 
  clip_cosine_similarities = tf.clip_by_value(cosine_similarities,-1.0,1.0) 
  scores = 1.0 - tf.acos(clip_cosine_similarities) / math.pi 
  return scores 

In [9]:
use_results = use_sts_benchmark(stsb['validation']) 
distilroberta_results = roberta_sts_benchmark(stsb['validation']) 

In [10]:
references = [item['label'] for item in stsb['validation']]

In [11]:
results = { 
      "USE":stsb_metric.compute( 
                predictions=use_results, 
                references=references), 
      "DistillRoberta":stsb_metric.compute( 
                predictions=distilroberta_results, 
                references=references) 
} 

In [12]:
import pandas as pd 
pd.DataFrame(results) 

,USE,DistillRoberta
pearson,0.810302,0.888461
spearmanr,0.808917,0.889246
